In [15]:
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
from colour.models import eotf_ST2084, eotf_inverse_ST2084

files = [
    (None, '/home/matt/dev/github/3ll3d00d/jrmc-utils/greyscale/greyscale_100_1000.png', 10000),
    (2.2, '/media/home-media/docs/calibration/tonemapping/greyscale/greyscale_jrvr_spline0.5_peak1000_3dlut_none.png', 1000),
    (2.2, '/home/matt/Pictures/orient_express.png', 1000),
    (2.2, '/home/matt/Pictures/orient_express_full.png', 1000),
    (2.2, '/media/home-media/showhorses.png', 1000),
    (None, '/media/home-media/showhorses.png', 1000),
    # (2.2, '/home/matt/Downloads/adl5.png')
]
data = []
for gamma, f, max_cll in files:
    im = np.array(Image.open(f).convert('RGB'))
    rgb = (im / 255) 
    linear_rgb = rgb ** gamma if gamma else rgb
    luminance = np.dot(linear_rgb[..., :3], [0.2126, 0.7152, 0.0722])
    nits_luminance = eotf_ST2084(luminance) 

    apl = np.mean(luminance) * 100
    fall = np.mean(nits_luminance)
    pq_fall = eotf_ST2084(np.mean(luminance))
    p9995 = np.percentile(nits_luminance, 99.95)
    pq_p9995 = eotf_ST2084(np.percentile(luminance, 99.95))
    max_nits = np.max(nits_luminance)
    max_xy = list(reversed(np.unravel_index(luminance.argmax(), luminance.shape)))
    data.append([Path(f).name, np.max(im), apl, fall, pq_fall, p9995, pq_p9995, max_nits, max_xy])

df = pd.DataFrame(data, columns=['Name', 'max(RGB)', 'APL %', 'FALL', 'PQ FALL', 'P99.95', 'PQ P99.95', 'MAX', 'MAX XY'])
df


,Name,max(RGB),APL %,FALL,EST FALL,PQ FALL,EST PQ FALL,P99.95,EST P99.95,PQ P99.95,EST PQ P99.95,MAX,MAX XY
0,greyscale_100_1000.png,192,37.933389,146.663321,148.360001,25.796814,25.794856,1010.272079,995.236944,1010.272079,909.775361,1010.272079,"(3762,)"
1,greyscale_jrvr_spline0.5_peak1000_3dlut_none.png,255,25.786805,471.791343,525.147557,5.751853,5.854604,10000.000000,9852.739402,10000.000000,8679.577610,10000.000000,"(3762,)"
2,orient_express_full.png,255,10.817709,18.397027,81.145625,0.399931,0.424496,1950.833360,1939.497634,1950.833360,2108.783546,9210.303265,"(200585,)"
3,showhorses.png,255,62.612736,1172.198560,1184.718675,312.277169,312.146731,7611.028205,7560.503579,7611.028205,8309.144118,10000.000000,"(1918225,)"
4,adl5.png,153,32.503696,13.693057,13.700869,13.693057,15.046635,13.693057,13.693065,13.693057,13.694360,13.693057,"(0,)"


In [18]:

eotf_ST2084(0.1077)

32.412353111715305